In [10]:
import pandas as pd

import sqlalchemy
from sqlalchemy import create_engine

In [11]:
# load data
pd.set_option('display.max_rows', 20)
stockCodes = ["AMZN", "AAPL", "META", "NFLX"]
stocks_df = pd.DataFrame()

# format the data correctly
for stockCode in stockCodes:
    df = pd.read_csv(f"Resources/{stockCode}-HistoricalData.csv")
    df = df.rename(columns={'Close/Last': 'Close'})
    df['StockCode'] = stockCode
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
    df['Close'] = df['Close'].str.replace('$', '')
    df['Open'] = df['Open'].str.replace('$', '')
    df['High'] = df['High'].str.replace('$', '')
    df['Low'] = df['Low'].str.replace('$', '')
    df = df.convert_dtypes()
    #display(df)
    stocks_df = pd.concat([stocks_df,df])

# convert the data types to be float for the numbers
stocks_df[['Close','Open','High','Low']] = stocks_df[['Close','Open','High','Low']].astype(float)
# convert the data to be correct
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'], format='%Y-%m-%d')

display(stocks_df.dtypes)
display(stocks_df.head())

Date         datetime64[ns]
Close               float64
Volume                Int64
Open                float64
High                float64
Low                 float64
StockCode    string[python]
dtype: object

,Date,Close,Volume,Open,High,Low,StockCode
0,2024-03-04,177.58,37381520,177.530,180.140,177.49,AMZN
1,2024-03-01,178.22,31981150,176.750,178.725,176.07,AMZN
2,2024-02-29,176.76,53805360,173.010,177.220,172.85,AMZN
3,2024-02-28,173.16,28180480,172.440,174.050,172.27,AMZN
4,2024-02-27,173.54,31141730,174.075,174.620,172.86,AMZN


In [12]:
engine = create_engine("sqlite:///stocks.sqlite")
stocks_df.to_sql('stocks', con=engine, if_exists='replace')

10068

In [13]:
stocks_from_sql_df = pd.read_sql_table('stocks', con=engine, parse_dates=['Date'])

stocks_from_sql_df['StockCode'] = stocks_from_sql_df['StockCode'].astype('string')
stocks_

display(stocks_from_sql_df.dtypes)
display(stocks_from_sql_df.head())

index                 int64
Date         datetime64[ns]
Close               float64
Volume                int64
Open                float64
High                float64
Low                 float64
StockCode    string[python]
dtype: object

,index,Date,Close,Volume,Open,High,Low,StockCode
0,0,2024-03-04,177.58,37381520,177.530,180.140,177.49,AMZN
1,1,2024-03-01,178.22,31981150,176.750,178.725,176.07,AMZN
2,2,2024-02-29,176.76,53805360,173.010,177.220,172.85,AMZN
3,3,2024-02-28,173.16,28180480,172.440,174.050,172.27,AMZN
4,4,2024-02-27,173.54,31141730,174.075,174.620,172.86,AMZN


In [14]:


stocks = dict()
stocks_monthly_df = None
stocks_monthly_df = stocks_from_sql_df.copy()
stocks_monthly_df = stocks_monthly_df.set_index('Date')
stocks_monthly_df = stocks_monthly_df.sort_index()
for stockCode in stockCodes: 
    stocks[stockCode+'_var']  = stocks_monthly_df.loc[stocks_monthly_df['StockCode'] == stockCode, 'Close':'Low'].resample('M').var()
    stocks[stockCode+'_mean'] = stocks_monthly_df.loc[stocks_monthly_df['StockCode'] == stockCode, 'Close':'Low'].resample('M').mean()

display(stocks['AAPL_mean'].head())
display(stocks['AAPL_var'].head())

,Close,Volume,Open,High,Low
Date,,,,,
2014-03-31,19.057353,2.364134e+08,19.080905,19.177153,18.949095
2014-04-30,19.324100,3.045542e+08,19.288648,19.441038,19.152233
2014-05-31,21.542724,2.723101e+08,21.475076,21.656195,21.369048
2014-06-30,22.971367,2.289699e+08,22.978162,23.142543,22.796776
2014-07-31,23.906395,1.868835e+08,23.932155,24.102600,23.709800


,Close,Volume,Open,High,Low
Date,,,,,
2014-03-31,0.031649,3.293795e+15,0.037944,0.033578,0.029346
2014-04-30,0.842856,2.436417e+16,0.727476,0.855401,0.755151
2014-05-31,0.288667,8.419495e+15,0.274585,0.310772,0.270646
2014-06-30,0.087722,7.770514e+15,0.109371,0.093367,0.085678
2014-07-31,0.176916,3.305682e+15,0.174375,0.165776,0.166150


In [15]:
%matplotlib inline

from ipywidgets import interact
import matplotlib.pyplot as plt

#plot the graph
def display_plot(valuation, statistical_method, current_stock):
    fig, ax = plt.subplots(figsize=(14,6))
    plt.title(f"{statistical_method} for {valuation} price vs Volume for {current_stock}")

    if statistical_method=='Variance':
        statistical_method = 'var'
    elif statistical_method=='Average':
        statistical_method = 'mean'

    stocks[current_stock+'_'+statistical_method].plot(y=[valuation], ax = ax, ylabel=valuation)
    stocks[current_stock+'_'+statistical_method].plot(y=['Volume'], ax = ax, secondary_y=True, ylabel='Volume')

    plt.show()

# make the graph interactive
interact(display_plot, 
         valuation=[
             'High',
             'Low',
             'Open',
             'Close',
         ],
         statistical_method = [
            'Variance',
            'Average',
         ],
         current_stock=[
             ('Amazon', 'AMZN'),
             ('Apple', 'AAPL'),
             ('Netflix', 'NFLX'),
             ('Meta', 'META'),
         ]);

interactive(children=(Dropdown(description='valuation', options=('High', 'Low', 'Open', 'Close'), value='High'…

In [16]:
#pd.__version__
%pip show pandas

Name: pandas
Version: 2.0.3
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: 
Author: 
Author-email: The Pandas Development Team <pandas-dev@python.org>
License: BSD 3-Clause License
        
        Copyright (c) 2008-2011, AQR Capital Management, LLC, Lambda Foundry, Inc. and PyData Development Team
        All rights reserved.
        
        Copyright (c) 2011-2023, Open source contributors.
        
        Redistribution and use in source and binary forms, with or without
        modification, are permitted provided that the following conditions are met:
        
        * Redistributions of source code must retain the above copyright notice, this
          list of conditions and the following disclaimer.
        
        * Redistributions in binary form must reproduce the above copyright notice,
          this list of conditions and the following disclaimer in the documentation
          and/or other materials provided with the distrib